In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from sklearn.datasets import fetch_california_housing

In [3]:
housing = fetch_california_housing()

In [4]:
housing.data.shape

(20640, 8)

In [5]:
housing.target.shape

(20640,)

In [6]:
num_examples, num_features = housing.data.shape

In [7]:
print(num_examples, num_features)

20640 8


In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
ss = StandardScaler()

In [10]:
scaled_housing_data = ss.fit_transform(housing.data)

In [11]:
scaled_housing_data_withbias = np.c_[np.ones((num_examples, 1)), scaled_housing_data]

In [12]:
scaled_housing_data_withbias.shape

(20640, 9)

In [13]:
num_examples, num_features = scaled_housing_data_withbias.shape

In [14]:
print(num_examples, num_features)

20640 9


Let's create tensorflow objects

In [15]:
x = tf.constant(scaled_housing_data_withbias, dtype=tf.float32)
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32)

Here are the weights/coefficients

In [16]:
theta = tf.Variable(tf.random_uniform([num_features, 1], -1.0, 1.0), dtype=tf.float32)

In [17]:
y_pred = tf.matmul(x, theta)

In [18]:
error = y_pred - y

In [19]:
mse = tf.reduce_mean(tf.square(error))

In [20]:
learning_rate = 0.01
num_epochs = 1000

In [21]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

In [22]:
training_op = optimizer.minimize(mse)

In [23]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [24]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    
    for e in range(num_epochs):
        if e % 100 == 0:
            print(mse.eval())
        sess.run(training_op)
        
    best_weights = theta.eval()
    print(best_weights)

7.11369
0.767844
0.623783
0.59519
0.576408
0.562776
0.552801
0.545482
0.5401
0.536132
[[  2.06855226e+00]
 [  7.24370956e-01]
 [  1.30277991e-01]
 [ -8.20849277e-03]
 [  6.72029182e-02]
 [  1.05572958e-03]
 [ -3.80570740e-02]
 [ -9.14687991e-01]
 [ -8.70265305e-01]]


In [43]:
idx = 2001

instance = tf.constant(scaled_housing_data_withbias[idx].reshape(-1, 1), dtype=tf.float32)
weights = tf.constant(best_weights.reshape(-1, 1), dtype=tf.float32)
y_hat = tf.matmul(tf.transpose(instance), best_weights)

with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    print("Predicted", y_hat.eval()[0][0])
print("Actual", housing.target[idx])

Predicted 0.737961
Actual 0.5
